# 03 Analysis of histone dynamics

## 1. Imports

In [15]:
import pandas as pd
import pickle as pkl

## 2. Load data

In [4]:
mRNA_file = "../data/FISH_RNAseq_combined_FACS_phases_7_genes_new_CC_phases.xlsx"

Relative mRNAs (Lotte's RNAseq data, normalized at each time point):

In [10]:
mRNA_rel = pd.read_excel(mRNA_file, sheet_name="nostress_WT_R1_normalized", skiprows=1, usecols='A, D:P')

Get systematic names from another sheet:

In [32]:
systematic_names = pd.read_excel(mRNA_file, sheet_name="nostress_WT_R1_absolute", skiprows=1, usecols='A', skipfooter=2)
systematic_names.rename(columns={"ORF (Yoav))": "systematic"}, inplace=True)

In [11]:
# time_tes = pd.read_pickle("../parameters/te_time-resolved.p") # pkl.load(open("../../parameters/te_time-resolved.p"))

In [12]:
# init_rates = pd.read_pickle("../parameters/init_rates_plotkin.p") # pkl.load(open("../../parameters/init_rates_plotkin.p"))

In [34]:
mRNA_rel['systematic'] = systematic_names['systematic']

In [37]:
mRNA_rel.head()

,Gene,0,5,10,15,20,25,30,35,40,45,50,55,60,systematic
0,TFC3,0.000013,0.000054,0.000032,0.000024,0.000029,0.000031,0.000026,0.000012,0.000030,0.000025,0.000017,0.000035,0.000036,YAL001C
1,VPS8,0.000042,0.000005,0.000025,0.000024,0.000021,0.000025,0.000030,0.000037,0.000023,0.000025,0.000022,0.000020,0.000027,YAL002W
2,EFB1,0.001254,0.000999,0.001135,0.001217,0.001211,0.001374,0.001378,0.001432,0.000990,0.001696,0.001355,0.001418,0.001401,YAL003W
3,YAL004W,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,YAL004W
4,SSA1,0.001864,0.001696,0.001471,0.001117,0.001181,0.001143,0.001278,0.001357,0.001487,0.001231,0.001527,0.001430,0.001339,YAL005C


In [40]:
histones = {'HHF1': 'YBR009C', 'HHF2': 'YNL030W', 
            'HHT1': 'YBR010W', 'HHT2': 'YNL031C', 
            'HTB1': 'YDR224C', 'HTB2': 'YBL002W', 
            'HTA1': 'YDR225W', 'HTA2': 'YBL003C'}

In [47]:
histones_systematic = list(histones.values())

## 3. Histone mRNAs

### 3.1 Relative amount of mRNAs (RNAseq data)

In [50]:
histones_df = mRNA_rel[mRNA_rel["systematic"].isin(histones_systematic)]

In [51]:
non_histones_df = mRNA_rel[~mRNA_rel["systematic"].isin(histones_systematic)]

### 3.2 Absolute amount of mRNAs

## 4. Histone proteins